In [1]:
import pandas
import pandasql
import numpy as np

df=pandas.read_csv('Historic_Secured_Property_Tax_Rolls.csv', low_memory=False) 
#replace header space by _ 
df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)


In [2]:
#q1: What fraction of assessments are for properties of the most common class? 
#For now, consider all the assessments, even though some properties may be listed more than once.
total_count=len(df['property_class_code'])
top_class_count=df['property_class_code'].value_counts()[0]
most_common_class_fraction=float(top_class_count)/float(total_count)
print "Fraction of most common class: %.10f" %most_common_class_fraction

Fraction of most common class: 0.4707253227


In [3]:
#q2:What is the median assessed improvement value, considering only non-zero assessments? 
#Consider only the latest assessment value for each property, which is uniquely identified by 
#the "Block and Lot Number" column.
q="""
SELECT block_and_lot_number,max(closed_roll_fiscal_year),closed_roll_assessed_improvement_value
FROM df
WHERE closed_roll_assessed_improvement_value>0
GROUP BY block_and_lot_number
HAVING count(block_and_lot_number)>1;
"""
sol=pandasql.sqldf(q.lower(), locals())
median_improvement_value=sol['closed_roll_assessed_improvement_value'].median()
print "Median assessed improvement value %.10f" % median_improvement_value

Median assessed improvement value 208500.0000000000


In [7]:
#create a non-duplicate df, take the latest assessed year
q="""
SELECT block_and_lot_number,max(closed_roll_fiscal_year),neighborhood_code,closed_roll_assessed_improvement_value,closed_roll_assessed_land_value,zipcode_of_parcel,property_area_in_square_feet,lot_area,number_of_rooms,number_of_units,year_property_built,location,property_location
FROM df
GROUP BY block_and_lot_number
HAVING count(block_and_lot_number)>1
"""
df2=pandasql.sqldf(q.lower(),locals())
df2=df2.rename(columns = {'max(closed_roll_fiscal_year)':'latest_closed_roll_fiscal_year'})


In [9]:
#q3:Calculate the average improvement value (using only non-zero assessments) in each neighborhood. 
#What is the difference between the greatest and least average values?
q="""
SELECT neighborhood_code,AVG(closed_roll_assessed_improvement_value)
FROM df2
WHERE closed_roll_assessed_improvement_value>0
GROUP BY neighborhood_code;
"""

sol2=pandasql.sqldf(q.lower(), locals())

mx2=max(sol2['avg(closed_roll_assessed_improvement_value)'])
mn2=min(sol2['avg(closed_roll_assessed_improvement_value)'])
diff2=(mx2-mn2)
print "Difference between the greatest and least average values: %.10f" % diff2

Difference between the greatest and least average values: 15681710.3048042636


In [16]:
#q4: What is the yearly growth rate of Land Values over the years covered by this data? 
#Take a simplistic model: the value is given by the exponential growth P=Po*exp(r*t), where t is measured in years. 
#(A more complete model would give each property its own base price Po. 
#Estimate r over all assessments with a non-zero land value. (Hint: Consider using linear regression and logarithms.)

q="""
SELECT block_and_lot_number,closed_roll_fiscal_year,closed_roll_assessed_land_value
FROM df
WHERE closed_roll_assessed_land_value>0;
"""

sol3=pandasql.sqldf(q.lower(), locals())

from sklearn import linear_model

x=sol3['closed_roll_fiscal_year']
y=sol3['closed_roll_assessed_land_value']
#make array
x_temp=x.values
x_array=x_temp.reshape((len(x_temp),1))
y_temp=y.values
y_array=np.log(y_temp)
regr = linear_model.LinearRegression()
regr.fit(x_array, y_array)
#print 'Coefficients: \t', regr.coef_ 
growth_rate=np.exp(regr.coef_)
print 'Growth rate (/yr): %.10f' %growth_rate

Growth rate (/yr): 1.0427696058


In [103]:
#q5: We can use the property locations to estimate the areas of the neighborhoods. 
#Represent each as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude. 
#What is the area, in square kilometers, of the largest neighborhood measured in this manner? 
#Be sure to filter out invalid coordinates.

#add two columns: longitude and latitude
loc=df2['location']
coord=loc.str[1:-1].str.split(', ')
df2['longitude']=coord.str[1]
df2['latitude']=coord.str[0]

#get average longitude and latitude in each neighborhood
q="""
SELECT neighborhood_code,AVG(longitude),AVG(latitude)
FROM df2
WHERE longitude IS NOT NULL AND latitude IS NOT NULL 
GROUP BY neighborhood_code;
"""
sol10=pandasql.sqldf(q.lower(),locals())
sol10=sol10.rename(columns = {'avg(longitude)':'avg_longitude'})
sol10=sol10.rename(columns={'avg(latitude)':'avg_latitude'})
#join two dataframes
q="""
SELECT df2.neighborhood_code,df2.longitude,df2.latitude,sol10.avg_longitude,sol10.avg_latitude
FROM sol10 join df2 
ON sol10.neighborhood_code=df2.neighborhood_code
WHERE longitude IS NOT NULL AND latitude IS NOT NULL; 
"""
sol11=pandasql.sqldf(q.lower(),locals())
#calculate residual squares of longitude and latitude
q="""
SELECT neighborhood_code,(longitude-avg_longitude)*(longitude-avg_longitude),(latitude-avg_latitude)*(latitude-avg_latitude)
FROM sol11;
"""
sol12=pandasql.sqldf(q.lower(),locals())
sol12=sol12.rename(columns={'(longitude-avg_longitude)*(longitude-avg_longitude)':'res_longitude'})
sol12=sol12.rename(columns={'(latitude-avg_latitude)*(latitude-avg_latitude)':'res_latitude'})
#calculate variance of each neighborhood
q="""
SELECT neighborhood_code,SUM(res_longitude)/(COUNT(res_longitude)-1),SUM(res_latitude)/(COUNT(res_latitude)-1)
FROM sol12
GROUP BY neighborhood_code;
"""
sol13=pandasql.sqldf(q.lower(),locals())
sol13=sol13.rename(columns={'sum(res_longitude)/(count(res_longitude)-1)':'var_longitude'})
sol13=sol13.rename(columns={'sum(res_latitude)/(count(res_latitude)-1)':'var_latitude'})
#1 degree latitude=111 km. 1 degree longitude=85 km near 40 degree.
import math
max_lon=max(sol13['var_longitude'])
max_lat=max(sol13['var_latitude'])
lon_km=np.sqrt(max_lon)*85.
lat_km=np.sqrt(max_lat)*111.
max_neighborhood_area=math.pi*lon_km*lat_km
print "Largest neighborhood area: %.10f" %max_neighborhood_area +'(km^2)'

Largest neighborhood area: 3.3131880752(km^2)


In [113]:
#q6: What is the difference between the average number of units in buildings build in or after 1950, 
#and that for buildings built before 1950? Consider only buildings that have non-zero units reported, 
#and ignore buildings with obviously incorrect years. 
#To try to avoid the effect of improvements to buildings, use the earliest record for each property, not the latest.

#build in or after 1950 
q1="""
SELECT block_and_lot_number,MIN(closed_roll_fiscal_year),year_property_built, number_of_units
FROM df
WHERE number_of_units>0 
GROUP BY block_and_lot_number
HAVING COUNT(block_and_lot_number)>1;
"""
sol9=pandasql.sqldf(q1.lower(), locals())
sol9=sol9.rename(columns = {'min(closed_roll_fiscal_year)':'earliest_closed_roll_fiscal_year'})
q2="""
SELECT block_and_lot_number,year_property_built, number_of_units
FROM sol9
WHERE year_property_built>=1950 AND year_property_built<2016;
"""
after=pandasql.sqldf(q2.lower(), locals())

q3="""
SELECT block_and_lot_number,year_property_built, number_of_units
FROM sol9
WHERE year_property_built<1950 AND year_property_built>1700;
"""
before=pandasql.sqldf(q3.lower(),locals())
AvgUnitBf50=float(sum(before['number_of_units']))/float(len(before))
AvgUnitAf50=float(sum(after['number_of_units']))/float(len(after))

print "Difference of average units after and before 1950: %.10f"%(AvgUnitAf50-AvgUnitBf50)

Difference of average units after and before 1950: 0.5356193402


In [114]:
#q7: Considering only properties with non-zero numbers of bedrooms and units, 
#calculate the average number of bedrooms per unit in each zip code. 
#Use the ratio of the means instead of the mean of the ratio. 
#What is this ratio in the zip code where it achieves its maximum?
q="""
SELECT zipcode_of_parcel,AVG(number_of_rooms),AVG(number_of_units)
FROM df2
WHERE number_of_rooms>0 AND number_of_units>0
GROUP BY zipcode_of_parcel;
"""

sol7=pandasql.sqldf(q.lower(), locals())
sol7['ratio']=sol7['avg(number_of_rooms)']/sol7['avg(number_of_units)']

print "Max ratio of rooms to units: %.10f" %max(sol7['ratio'])

Max ratio of rooms to units: 24.2869198312


In [116]:
#q8: Estimate how built-up each zip code is by comparing the total property area to the total lot area. 
#What is the largest ratio of property area to surface area of all zip codes?

q="""
SELECT zipcode_of_parcel, SUM(property_area_in_square_feet),SUM(lot_area)
FROM df2
WHERE property_area_in_square_feet>0 AND lot_area>0
GROUP BY zipcode_of_parcel;
"""

sol5=pandasql.sqldf(q.lower(), locals())
sol5['build_up']=sol5['sum(property_area_in_square_feet)']/sol5['sum(lot_area)']

print 'Max build_up for all zipcodes: %.10f'% max(sol5['build_up'])


Max build_up for all zipcodes: 13.5872646159
